In [1]:
# ROAD ACCIDENT DATA CLEANING

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load the dataset
df = pd.read_csv(r"C:\Users\Mounika\Desktop\Final Project\Dataset\Indian_Road_Accidents_(2018-2023).csv")

In [4]:
# Preview
print("Data Loaded Successfully!")
print(df.shape)
print(df.head())

Data Loaded Successfully!
(3000, 36)
          State Name City Name  Year    Month Day of Week Time of Day  \
0  Jammu and Kashmir   Unknown  2021      May      Monday        1:46   
1      Uttar Pradesh   Lucknow  2018  January   Wednesday       21:30   
2       Chhattisgarh   Unknown  2023      May   Wednesday        5:37   
3      Uttar Pradesh   Lucknow  2020     June    Saturday        0:31   
4             Sikkim   Unknown  2021   August    Thursday       11:21   

  Accident Severity  Number of Vehicles Involved Vehicle Type Involved  \
0           Serious                            5                 Cycle   
1             Minor                            5                 Truck   
2             Minor                            5            Pedestrian   
3             Minor                            3                   Bus   
4             Minor                            5                 Cycle   

   Number of Casualties  ...  Vehicle Type Involved_encoded  \
0               

In [5]:
# Basic Cleaning

# Strip whitespace from column names
df.columns = df.columns.str.strip()

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Fill or remove missing values
df['City Name'] = df['City Name'].fillna('Unknown')
df['Weather Conditions'] = df['Weather Conditions'].fillna('Clear')
df['Road Condition'] = df['Road Condition'].fillna('Unknown')
df['Accident Severity'] = df['Accident Severity'].fillna('Unknown')

# Drop rows where critical data like 'Year' or 'State Name' is missing
df = df.dropna(subset=['Year', 'State Name'])

# Convert data types if needed
df['Year'] = df['Year'].astype(int, errors='ignore')


Missing values per column:
State Name                            0
City Name                             0
Year                                  0
Month                                 0
Day of Week                           0
Time of Day                           0
Accident Severity                     0
Number of Vehicles Involved           0
Vehicle Type Involved                 0
Number of Casualties                  0
Number of Fatalities                  0
Weather Conditions                    0
Road Type                             0
Road Condition                        0
Lighting Conditions                   0
Traffic Control Presence            716
Speed Limit (km/h)                    0
Driver Age                            0
Driver Gender                         0
Driver License Status               975
Alcohol Involvement                   0
Accident Location Details             0
hour                                  0
State Name_encoded                    0
City Name_en

In [6]:
# Phase column based on year
def phase_map(year):
    if year in [2018, 2019]:
        return '2018–2019'
    elif year in [2020, 2021]:
        return '2020–2021'
    elif year in [2022, 2023]:
        return '2022–2023'
    else:
        return 'Other'

df['Phase'] = df['Year'].apply(phase_map)

print("\nAccidents per phase:")
print(df['Phase'].value_counts())


Accidents per phase:
Phase
2020–2021    1048
2022–2023    1005
2018–2019     947
Name: count, dtype: int64


In [7]:
# Quick Summary Analysis
accidents_per_year = df.groupby('Year').size()
accidents_per_phase = df.groupby('Phase').size()
fatalities_phase = df.groupby('Phase')['Number of Fatalities'].sum()
severity_phase = df.groupby(['Phase', 'Accident Severity']).size().unstack(fill_value=0)

print("\nAccidents per Year:")
print(accidents_per_year)
print("\nAccidents per Phase:")
print(accidents_per_phase)
print("\nFatalities per Phase:")
print(fatalities_phase)
print("\nAccident Severity by Phase:")
print(severity_phase)


Accidents per Year:
Year
2018    479
2019    468
2020    526
2021    522
2022    521
2023    484
dtype: int64

Accidents per Phase:
Phase
2018–2019     947
2020–2021    1048
2022–2023    1005
dtype: int64

Fatalities per Phase:
Phase
2018–2019    2291
2020–2021    2660
2022–2023    2415
Name: Number of Fatalities, dtype: int64

Accident Severity by Phase:
Accident Severity  Fatal  Minor  Serious
Phase                                   
2018–2019            289    329      329
2020–2021            347    357      344
2022–2023            349    348      308


In [8]:
# Drop unwanted encoded columns if present
encoded_cols = [
    'State Name_encoded', 'City Name_encoded', 'Accident Severity_encoded',
    'Vehicle Type Involved_encoded', 'Weather Conditions_encoded', 'Road Type_encoded',
    'Road Condition_encoded', 'Lighting Conditions_encoded', 'Traffic Control Presence_encoded',
    'Driver Gender_encoded', 'Driver License Status_encoded', 'Alcohol Involvement_encoded'
]

df = df.drop(columns=[col for col in encoded_cols if col in df.columns], errors='ignore')

In [9]:
# Fill missing categorical values
cols_to_fill = [
    'City Name', 'Weather Conditions', 'Road Condition', 
    'Accident Severity', 'Vehicle Type Involved', 'Road Type',
    'Lighting Conditions', 'Traffic Control Presence', 
    'Driver Gender', 'Driver License Status', 'Alcohol Involvement',
    'Accident Location Details'
]

for col in cols_to_fill:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

In [10]:
# Derived Columns (for dashboard visualization)

# Unique Accident ID
df.insert(0, 'Accident_ID', ['A{:05d}'.format(i) for i in range(1, len(df) + 1)])

# Weekend / Weekday
if 'Day of Week' in df.columns:
    df['Weekend'] = df['Day of Week'].isin(['Saturday', 'Sunday']).map({True: 'Weekend', False: 'Weekday'})
else:
    df['Weekend'] = 'Unknown'

# Fatal / Non-Fatal Flag
if 'Number of Fatalities' in df.columns:
    df['Is_Fatal'] = df['Number of Fatalities'].apply(lambda x: 'Fatal' if x > 0 else 'Non-Fatal')
else:
    df['Is_Fatal'] = 'Unknown'

# Fatality Rate (per accident)
if 'Number of Fatalities' in df.columns and 'Number of Casualties' in df.columns:
    df['Fatality_Rate'] = (df['Number of Fatalities'] / df['Number of Casualties']).replace([np.inf, np.nan], 0).round(2)
else:
    df['Fatality_Rate'] = 0

# Time Bin (based on hour)
if 'hour' in df.columns:
    def time_bin(h):
        try:
            h = int(h)
        except:
            return 'Unknown'
        if 5 <= h < 12: return 'Morning'
        elif 12 <= h < 17: return 'Afternoon'
        elif 17 <= h < 21: return 'Evening'
        elif 21 <= h or h < 5: return 'Night'
        return 'Unknown'
    df['Time_Bin'] = df['hour'].apply(time_bin)

# Driver Age Group
if 'Driver Age' in df.columns:
    bins = [0, 18, 25, 35, 50, 65, 120]
    labels = ['<18','18-24','25-34','35-49','50-64','65+']
    df['Driver_Age_Group'] = pd.cut(df['Driver Age'], bins=bins, labels=labels, right=False)

# Speed Band
if 'Speed Limit (km/h)' in df.columns:
    bins = [0, 30, 50, 70, 90, 110, 200]
    labels = ['0–29','30–49','50–69','70–89','90–109','110+']
    df['Speed_Band'] = pd.cut(df['Speed Limit (km/h)'], bins=bins, labels=labels, right=False)


In [ ]:
# Export Cleaned Data 

output_path = r"C:\Users\Mounika\Desktop\final Project\Dataset\Cleaned_Data_Set.xlsx"
df.to_excel(output_path, index=False)
print(f"\nCleaned dataset with derived columns saved to: {output_path}")

In [12]:
# Final Check

print("\nFinal Columns:")
print(df.columns.tolist())
print(f"\nFinal shape: {df.shape}")


Final Columns:
['Accident_ID', 'State Name', 'City Name', 'Year', 'Month', 'Day of Week', 'Time of Day', 'Accident Severity', 'Number of Vehicles Involved', 'Vehicle Type Involved', 'Number of Casualties', 'Number of Fatalities', 'Weather Conditions', 'Road Type', 'Road Condition', 'Lighting Conditions', 'Traffic Control Presence', 'Speed Limit (km/h)', 'Driver Age', 'Driver Gender', 'Driver License Status', 'Alcohol Involvement', 'Accident Location Details', 'hour', 'Time_Bin', 'Phase', 'Weekend', 'Is_Fatal', 'Fatality_Rate', 'Driver_Age_Group', 'Speed_Band']

Final shape: (3000, 31)
